In [32]:
import time 
import numpy as np
import ipyparallel as ipp
from svm import SVM
from AggregatorFunc import snd_to_eng, wc_from_eng, global_update, bg_from_eng, belta_update, grad_update, delta_update, binary_search

In [33]:
c = ipp.Client()

In [34]:
dview = c[:]

In [35]:
import os
dview.map(os.chdir, ['D:/GitHub/Parallel/classifier']*len(c.ids))
print(dview.apply_sync(os.getcwd))

['D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier', 'D:\\GitHub\\Parallel\\classifier']


In [76]:
from keras.datasets import mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [77]:
train_feature = x_train.reshape(x_train.shape[0],-1,)
train_label = y_train

In [79]:
#Label training targets by even(1) and odd(-1)
train_target = np.zeros(train_label.shape)
for i in range(len(train_target)):
    if train_label[i]%2 == 0:
        train_target[i] = 1
    else:
        train_target[i] = -1

In [81]:
#sending date to engines
c[0]['X'] = train_feature[:15000]
c[1]['X'] = train_feature[15000:30000]
c[2]['X'] = train_feature[30000:45000]
c[3]['X'] = train_feature[45000:]

c[0]['y'] = train_target[:15000]
c[1]['y'] = train_target[15000:30000]
c[2]['y'] = train_target[30000:45000]
c[3]['y'] = train_target[45000:]

In [115]:
t = 0
s = 0
b = 0 # aggregator consumption
R = 15

data_size = np.array([15000,15000,15000,15000])
gamma = 10
phi = 0.2
w_aggregator = np.zeros(len(train_feature[0]))
torque_aggregator = 1
stop = False

In [116]:
svm = SVM()
dview.push(dict(SVM = SVM))
#sending LinearRegression object to engines
dview['svm'] = svm

In [118]:
while True:
        tic = time.time()
        snd_to_eng(w_aggregator,torque_aggregator,c)
        t_0 = t
        t = t + torque_aggregator
        dview.execute("""
import numpy as np
import time    
svm.Rec_from_Agg(w_aggregator, torque_aggregator)
svm.time_record()
if svm.t > 0:
    svm.Est_Belta(X,y)
svm.fit(X,y)
        """)
        
        
        w_local, c_local = wc_from_eng(c,'svm')
        #calculate local consumption per iteration
        c_per = np.array(c_local)/torque_aggregator
        w_aggregator = global_update(w_local, data_size)
        if stop:
            w_final = w_aggregator
            break
        #c_local.sum() equal to c*t
        s = s + np.array(c_local).sum()+ b
        
        if t_0 > 0:
            belta_local, grad_local = bg_from_eng(c,'svm')
            belta_aggregator = belta_update(belta_local,data_size)
            grad_aggregator = grad_update(grad_local,data_size)
            delta_aggregator = delta_update(grad_local, grad_aggregator, data_size)
            torque_aggregator, G_list = binary_search(torque_aggregator, delta_aggregator, belta_aggregator, gamma, phi)
            print('New torque is:',torque_aggregator)
        
        toc = time.time()
        b = toc - tic
        temp = s + torque_aggregator*c_per.sum() + b
        if temp >= R:
            torque_max = (R-b-s)/c_per.sum()
            G_list = np.array(G_list)
            G_min = G_list.min()
            for i, item in enumerate(G_list):
                if item >= torque_max:
                    itme = G_min
            torque_aggregator = np.argmax(G_list) + 1
            stop = True 

New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1
New torque is: 1


C:\Users\wenbi\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars


In [119]:
pred = np.dot(train_feature,w_final)

In [120]:
pred[:10]

array([-7320973.5724043 ,  2026368.66405463, -1211849.86096836,
       -1615523.04006576, -3093949.35069831,   223289.21982547,
       -5984845.85745675, -3957271.52129634, -3940302.35308118,
        2630924.55430885])

In [121]:
for i in range(len(pred)):
    if pred[i]>0:
        pred[i]= 1
    else:
        pred[i] = -1

In [122]:
flag=0
for i in range(len(pred)):
    if pred[i] - train_target[i] != 0:
        flag+=1

In [123]:
flag/60000

0.14991666666666667